### 1. Complete the sentence: The geometric distribution is a special case of the negative binomial distribution where ___

- r = 1

### 2. Calculate the following probabilities.
- You toss a coin 4 times. The probability that you get (exactly) 2 heads.
- You toss a coin until you get 2 heads. The probability that it takes (exactly) 4 tosses.

- You toss a coin 4 times. The probability that you get (exactly) 2 heads.
    - Binomial
        - $$
            \binom{4}{2} 0.5^{2} \cdot 0.5^{2} = 0.375
            $$

- You toss a coin until you get 2 heads. The probability that it takes (exactly) 4 tosses.
    - Negative binomial 
        - $$
            \binom{4-1}{2-1} 0.5^{2} \cdot 0.5^{2} = 0.1875
            $$

- Proof by simulation below

- The first is larger.
    - Intuitively, in the first case, getting 2 heads in 4 tosses allows you to arrange the heads however you like. i.e. HHTT is a valid combination, HTHT is a valid combination
    - Conversely, in the second case, you must have 2 heads in 4 tosses AND your 4th toss must be a head
        - So the only valid combinations are HTTH, THT, TTHH
    - Hence, the first must be larger, because the second is a subset of the first

In [4]:
import scipy
import numpy as np
print(scipy.stats.binom.pmf(n=4, p=0.5, k=2)) #n is the number of tosses, p is the probability of heads, k is the number of heads
print(scipy.stats.nbinom.pmf(n=2, p=0.5, k=2)) #n is the number of heads, k is the number of tails, p is the probability of heads


0.375
0.18750000000000006


In [5]:
population=[0] + [1]
np.mean([np.sum(np.random.choice(population, size=4, replace=True))==2 for _ in range(20_000)])

0.37775

In [7]:
population=[0] + [1]
n=20_000
counter=0
for _ in range(n):
    sample=np.random.choice(population, size=4, replace=True)
    if (sum(sample)==2) & (sample[-1]==1):
        counter+=1
print(counter/n)

0.18865


### 3. In Major League Baseball’s Home Run Derby, each contestant is allowed to keep swinging the bat until they have made 10 “outs”. (An “out” is anything that is not a home run.) If Barry Bonds has a 70% chance of hitting a home run on any given swing, what is the probability that he hits at least 10 home runs before his turn is up?

- Let outs be 1s, home runs be 0s, and swings be draws
- Let X be the total number of draws (swings) if BB draws (swings) up to 10 1s (outs) 
- X~NB(n=10, p=0.3), where n is the number of 1s (outs) before seqeunce ends, and p is the probability of making an out
- To have at least 10 home runs before striking out, BB has to swing a minimum of 20 times (10 to strike out, and the 10 home runs)
    - So probability of hitting at least 10 home runs is $P(X >= 20) = 1 - P(X <= 19) = 1 - F(19) = 1 - f(19) - f(18) ...$

    - $f(19) = P(X = 19) = \binom{19-1}{10-1} * 0.3^{10} * 0.7^{9} = 0.01158$
    - $f(18) = P(X = 18) = \binom{18-1}{10-1} * 0.3^{10} * 0.7^{8} = 0.00827$
    - ...


In [19]:
from math import comb
import scipy
print(comb(18, 9) * 0.3**10 * 0.7**9)
print(scipy.stats.nbinom.pmf(n=10, k=9, p=0.3)) ##equal

0.011585368759430456
0.011585368759430455


In [39]:
sum_f1_to_f19 = np.sum(scipy.stats.nbinom.pmf(n=10, k=range(10), p=0.3))
cdf_f1_to_f19 = scipy.stats.nbinom.cdf(k=9, n=10, p=0.3)
print(1 - sum_f1_to_f19) #probability of at least 10 home runs before striking out
print(1 - cdf_f1_to_f19)

0.9674466431186991
0.9674466431186991


In [32]:
population = [0]*7 + [1]*3

def make_sample():
    out_counter=0
    homerun_counter=0
    while out_counter < 10:
        val=np.random.choice(population, size=1, replace=True)[0]
        if val==0:
            homerun_counter+=1
        else:
            out_counter+=1
    return homerun_counter >= 10

np.mean([make_sample() for _ in range(10_000)])

0.9697

### 4. A medical researcher is recruiting 20 subjects for a study on an experimental drug for COVID-19. Each person that she interviews has a 60% chance of being eligible to participate in the study. What is the probability that she will have to interview more than 40 people?

- X~NB(n = 20, p = 0.6)
- Pr(X > 40) = 1 - Pr(X <= 40)

In [43]:
import scipy
import numpy as np
1 - np.sum(scipy.stats.nbinom.pmf(n=20, k=range(0,21), p=0.6))

0.07435155675691418

In [44]:
population = [1]*6 + [0]*4
def make_sample():
    recruited=0
    interviewed=0
    while recruited < 20:
        sampled=np.random.choice(population, size=1, replace=True)
        interviewed+=1
        if sampled==1:
            recruited+=1
    return interviewed > 40
np.mean([make_sample() for _ in range(10_000)])

0.07505

### 5. Your coach tells you that you cannot leave basketball practice until you have made at least 20 free throws. If you free throw probability is 80%, find the probability that you are out of practice after taking an even amount of free throws.

- 50%.. by intuition? lol

- p=0.8, n=20
- X~NB(n=20, p=0.8)
- P(X % 2 == 0)??

In [47]:
import scipy
import numpy as np

population=[1]*4 + [0]*1

def make_sample():
    scored=0
    attempt=0
    while scored < 20:
        shot=np.random.choice(population, size=1, replace=True)
        attempt+=1
        if shot==1:
            scored+=1
    return attempt % 2 == 0

np.mean([make_sample() for _ in range(20_000)])

0.5016

### 6. You have two coins. One coin is a fair coin with a .5 probability of landing on heads. The other coin is a biased coin with a .25 probability of landing on heads. You pick one of these two coins at random, and begin flipping until you get 5 heads. It takes you 12 flips in order to get your 5 heads. What is the probability that the coin you picked was the fair coin? What is the probability you picked the biased coin?

- Assume you have the fair coin
    - X~NB(n=5, p=0.5)
    - You want to know P(X=12)=f(12)
    - $\binom{12-1}{5-1} \cdot 0.5^{5} \cdot 0.5^{7}$
    - Let this be A

- Assume you have the biased coin
    - X~NB(n=5, p=0.25)
    - You want to know P(X=12)=f(12)
    - $\binom{12-1}{5-1} \cdot 0.25^{5} \cdot 0.75^{7}$
    - Let this be B

- Probability of biased coin is B/(A+B)

- Proof by simulation below

In [67]:
from math import comb
fair_coin_f12=comb(11, 4) * 0.5**12 
bias_coin_f12=comb(11, 4) * 0.25**5 * 0.75**7 

fair_coin_f12/(fair_coin_f12+bias_coin_f12)

0.6519178736272482

In [70]:
from collections import Counter
coin_population=[0.5] + [0.25]
coinflip_population={
    "0.5": [1] + [0],
    "0.25": [1] + [0]*3
}

def get_sample():
    choose_coin=str(np.random.choice(coin_population, size=1, replace=True)[0])
    sample=np.random.choice(coinflip_population.get(choose_coin), size=12, replace=True)
    if (np.sum(sample) == 5) & (sample[-1] == 1):
       return choose_coin
    
counts=Counter([get_sample() for _ in range(20_000)])
counts['0.5']/(counts['0.5'] + counts['0.25'])

0.6529266281945589